In [1]:
import pandas as pd
import openpyxl as exc
from openpyxl import workbook as wb
from openpyxl import load_workbook
import xlrd
import pyexcel as p
import os
from io import StringIO
import glob

In [47]:
def job_date(df):
    job = str(df.columns[0]).split(' ', 1)[1]
    edit_date = pd.to_datetime(str(df.iloc[0,0]).split(' ')[0], errors='coerce')
#     attendance = str(df.iloc[0,0]).split(' ',1)[1]
    return job, edit_date



In [60]:
completed_path = r"C:\Users\willd\OneDrive\Desktop\Belair\projection_analysis\completed proj projections"
c_recents = r"C:\Users\willd\OneDrive\Desktop\Belair\projection_analysis\c_recent\compiled_data.xlsx"
on_going_path = ''

In [5]:
files = []
for item in os.listdir(completed_path):
    job_code = item.split(' ')[1]
    files.append(job_code)
files= pd.unique(files)
    
#     full_path = os.path.join(completed_path, item)

In [7]:
latest_file_list = []
for file in files:
    job_files = [os.path.join(completed_path, x) for x in os.listdir(completed_path) if file in x]
    latest_file = max(job_files, key=os.path.getctime)
    latest_file_list.append(latest_file)
latest_file_list = pd.unique(latest_file_list)   

In [49]:
df_list = []
for item in latest_file_list:
    num_sheet = len(pd.ExcelFile(item).sheet_names)
    for sheet in range(0,num_sheet):
        xl = pd.read_excel(item ,sheet_name=sheet)
        df = pd.DataFrame(xl)
        job, date = job_date(df)
        df.columns = df.iloc[2]
        df = df.drop(df.index[:3])
        df = df.iloc[:, :27]
        df['updated'] = date
        df['job'] = job
#         df['attend'] = atend
        df['HJ Qty Comp'] = df['HJ Qty Comp'].fillna(0)
        df = df.dropna(subset=['Qty'])
        df = df.reset_index(drop=True)
#         df = df.loc[:,~df.columns.duplicated()].copy()
#         df = df.drop(df.columns[[26,27,28,29]], axis=1)
#         print(job)
#         print(job)
        #         print(date)
#         df = df.reset_index()
        df_list.append(df)
# full_df = pd.concat(df_list).drop_duplicates().reset_index(drop=True)
        #         print(df.head(3))

In [50]:
full_df = pd.concat(df_list).drop_duplicates().reset_index(drop=True)

In [39]:
list1 = df_list[0].columns.to_series()
list2 = pd.unique(list1)
print(len(list1), len(list2))

31 31


In [66]:
most_recent = full_df.sort_values(by=['job', 'Phase Code'])
most_recent = most_recent[~most_recent[['Qty','HJ Qty Comp']].eq(0).sum(1).ge(2)]
# df.drop_duplicates(subset=['IDName'], keep='last')


In [65]:
writer = pd.ExcelWriter(c_recents, engine="xlsxwriter")
most_recent.to_excel(writer, sheet_name="mk1")
# monthly_ratio.to_excel(writer, sheet_name="all_mon")
# money_in.to_excel(writer, sheet_name="income")
# money_out.to_excel(writer, sheet_name="expense")
writer.close()

In [23]:
import collections
print([item for item, count in collections.Counter(list1).items() if count > 1])

['% Comp', 'Budget Remaining', 'Over/ (Under) Budget']


In [40]:
df_list = []
for item in os.listdir(completed_path):
    full_path = os.path.join(completed_path, item)
    num_sheet = len(pd.ExcelFile(full_path).sheet_names)
    for sheet in range(0,num_sheet):
        
        xl = pd.read_excel(full_path ,sheet_name=sheet)
        df = pd.DataFrame(xl)
        df.columns.values[0] = "1"
#         df['1'] = df['1'].fillna(False)
        pc_list = df.index[df['1'].str.contains('Phase Code', na= False)].tolist()
        if len(pc_list) > 0:
            pc_ind = pc_list[0]
#         if 'Phase Code' in df.iloc[:,0]:   
#             job, date = job_date(df)
#             df.columns = df.iloc[pc_ind]
#             df = df.drop(df.index[:3])
#             df['updated'] = date
#             df['job'] = job
#             # remove non numeric values in this column
#             df = df[df['Phase Code'].apply(lambda x: str(x).isnumeric())]
#             df['HJ Qty Comp'] = df['HJ Qty Comp'].fillna(0)
            df_list.append(df)
full_df = pd.concat(df_list)    

